In [1]:
%pip install neo4j

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import llm_calls as lc
from time import sleep
%pip install faiss-cpu


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from neo4j import GraphDatabase, basic_auth, Config
import ssl
import logging
uri = "bolt://c24a04a8.databases.neo4j.io:7687"
username = "neo4j"
password = "TIEXesTzQ3V72Od9m-64Kpvd6AxVw8uB4fyvYODD_Jk"

# Create a custom SSL context that skips verification
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# Connect to Neo4j using the custom SSL context
driver = GraphDatabase.driver(uri, auth=basic_auth(username, password), encrypted=True, ssl_context=ssl_context)

def create_node(tx, label, properties):
    query = f"CREATE (n:{label} {{ {', '.join([f'{k}: ${k}' for k in properties.keys()])} }})"
    tx.run(query, **properties)

def create_relationship(tx, start_label, start_properties, end_label, end_properties, relationship):
    start_query = f"MATCH (a:{start_label} {{ {', '.join([f'{k}: ${k}' for k in start_properties.keys()])} }})"
    end_query = f"MATCH (b:{end_label} {{ {', '.join([f'{k}: ${k}' for k in end_properties.keys()])} }})"
    rel_query = f"CREATE (a)-[:{relationship}]->(b)"
    query = f"{start_query} {end_query} {rel_query}"
    tx.run(query, **{**start_properties, **end_properties})

try:
    with driver.session() as session:
        session.write_transaction(create_node, "Person", {"name": "Alice", "age": 30})
        session.write_transaction(create_node, "Person", {"name": "Bob", "age": 25})
        session.write_transaction(create_relationship, "Person", {"name": "Alice"}, "Person", {"name": "Bob"}, "KNOWS")
except Exception as e:
    logging.error(f"Error connecting to Neo4j: {e}")
finally:
    if driver:
        driver.close()


C:\Users\Abhay\AppData\Local\Temp\ipykernel_3516\1341383620.py:1: DeprecationWarning: Importing Config from neo4j is deprecated without replacement. It's internal and will be removed in a future version.
  from neo4j import GraphDatabase, basic_auth, Config
C:\Users\Abhay\AppData\Local\Temp\ipykernel_3516\1341383620.py:29: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Person", {"name": "Alice", "age": 30})
C:\Users\Abhay\AppData\Local\Temp\ipykernel_3516\1341383620.py:30: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Person", {"name": "Bob", "age": 25})
C:\Users\Abhay\AppData\Local\Temp\ipykernel_3516\1341383620.py:31: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, "Person", {"name": "Alice"}, "Person", {"name": "Bob"}, "KNOWS")


In [5]:
import faiss
import numpy as np

# Sample data for FAISS
data = np.array([
    [1.0, 2.0, 3.0],
    [2.0, 3.0, 4.0],
    [3.0, 4.0, 5.0],
]).astype('float32')

# Create the FAISS index
index = faiss.IndexFlatL2(data.shape[1])  # L2 distance

# Add data to the index
index.add(data)

# Function to search in FAISS index
def search_faiss(query_vector, k=5):
    distances, indices = index.search(np.array([query_vector]).astype('float32'), k)
    return indices, distances

# Example query
query_vector = [1.0, 2.0, 3.0]
print(search_faiss(query_vector))




(array([[ 0,  1,  2, -1, -1]], dtype=int64), array([[0.0000000e+00, 3.0000000e+00, 1.2000000e+01, 3.4028235e+38,
        3.4028235e+38]], dtype=float32))


In [ ]:

from transformers import pipeline

retriever = pipeline("feature-extraction", model="bert-base-uncased", tokenizer="bert-base-uncased")


def rag_pipeline(query):
    # Extract features for the query
    query_vector = retriever(query)[0]

    # Retrieve relevant data using FAISS
    indices, distances = search_faiss(query_vector, k=5)
    context = " ".join([str(data[idx]) for idx in indices[0]])

    # Generate response using the context
    response = lc.llm_generate(f"Question: {query} Context: {context}", max_length=512)
    return response[0]['generated_text']

# Example usage
query = "Tell me about Alice"
print(rag_pipeline(query))



d:\abhay\Work\projects\test\rag-pipeline\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 